In [28]:
import glob
import codecs
import re
from collections import Counter
from collections import defaultdict

import pandas as pd
import numpy as np
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/emersonsjsu/GitHub/EarlGrey/google_private_key.json
WORKING_DIRECTORY = "/Users/emersonsjsu/GitHub/EarlGrey"


env: GOOGLE_APPLICATION_CREDENTIALS=/Users/emersonsjsu/GitHub/EarlGrey/google_private_key.json


In [27]:
# Clear char_df
FIRST_EPISODE_OFFSET = 101
LAST_EPISODE_IN_SEASON = {25: 1, 47: 2, 73: 3, 99: 4, 125: 5, 151: 6, 176: 7}
regex = re.compile('[^a-zA-Z ]')


def get_season(episode_no):
    return LAST_EPISODE_IN_SEASON.get(episode_no, 0)


def sanitize_name(name_line):
    name = name_line.strip()  # Voice overs count as same character
    if name.endswith('V.O.'):
        name = name[:-5]
    # Strip parentheticals from name
    if name.find('(') != -1:
        name = name[:name.find('(')]
    # Strip 'S VOICE from name
    if name.endswith('\'S VOICE'):
        name = name[:-8]
    # Strip 'S COM VOICE from name
    if name.endswith('\'S COM VOICE'):
        name = name[:-12]
    return name.strip()


# counters_dict expects {char: [counter_obj, [s1count,s2count, ..., s7count]]}
def process_script(file_path, counters_dict):
    current_file = codecs.open(file_path, "r", encoding='utf-8', errors="ignore")
    episode_no = int(current_file.name[current_file.name.rfind('/') + 1:-4]) - FIRST_EPISODE_OFFSET
    is_dialogue = False
    character_name = ""
    lines = current_file.readlines()
    for line in lines:
        # Character dialogue has ended, reset current character
        if line.strip() == '':
            is_dialogue = False
        # All names are preceded by exactly 5 tabs
        elif line[:5] == '\t\t\t\t\t' and line[5:6] != '\t' and not line.strip().startswith('('):
            # We have found a name! Update current name
            character_name = sanitize_name(line)
            is_dialogue = True
        # If we are still looking at a character's dialogue
        elif is_dialogue:
            # Update character's counter object
            counters_dict[character_name][0].update(line.lower().split())
    #text_array = map(lambda x: ''.join(x)[1:], text_array)
    current_file.close()

    # Check if we are end of a season
    season_no = get_season(episode_no)
    if season_no:
        # Update wc per season for each character in counters_dict
        for v in counters_dict.itervalues():
            v[1][season_no - 1] = sum(v[0].values())


counters_dict = defaultdict(lambda: [Counter(), [0] * 7])
# for file_path in glob.glob(WORKING_DIRECTORY+'/scripts/225.txt'):  # Only read one file for debug purposes
for file_path in glob.glob(WORKING_DIRECTORY + '/scripts/*.txt'):
    process_script(file_path, counters_dict)

counters_dict['PICARD'][0]


Counter({u'considered.': 1,
         u'42372.5.': 1,
         u'foul': 1,
         u'four': 69,
         u'corridor...': 1,
         u'(touches': 14,
         u'ocean.': 4,
         u'hanging': 1,
         u'aggression': 1,
         u'increase': 25,
         u'course...': 11,
         u'specifics.': 2,
         u'specifics,': 1,
         u'liaison,': 3,
         u'felicium.': 3,
         u'maxwell...': 3,
         u'locutus...': 1,
         u'ruffled)': 1,
         u'advice.': 1,
         u'advice,': 2,
         u'tantalizing': 1,
         u'malfunctioned?': 1,
         u'spider?': 1,
         u'stabbed': 1,
         u'bringing': 8,
         u'data.)': 1,
         u'disturb': 2,
         u'wooden': 1,
         u'vortex.': 1,
         u'measure...': 1,
         u'path': 6,
         u'seven.': 9,
         u'seven,': 4,
         u'several...': 1,
         u'27,': 1,
         u'monitor,': 1,
         u'meantime...': 1,
         u'admitted.': 1,
         u'sustaining': 1,
         u'datas.'